In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
from sklearn.metrics import precision_score, f1_score, confusion_matrix
import seaborn as sns
import os
import sys
from datetime import datetime

Select device and using logger for report

In [3]:
# Base directory for storing all experiment results
base_output_dir = "experiment_results"
os.makedirs(base_output_dir, exist_ok=True)

def create_experiment_dir(base_dir=base_output_dir):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    exp_dir = os.path.join(base_dir, f"experiment_{timestamp}")
    os.makedirs(exp_dir, exist_ok=True)  # Ensure directory creation is safe
    return exp_dir

class Logger:
    def __init__(self, filename):
        self.terminal = sys.stdout
        self.log = open(filename, "a")  # Append mode to keep previous logs

    def write(self, message):
        self.terminal.write(message)
        self.log.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}")  # Add timestamp
        self.flush()

    def flush(self):
        # Ensure both streams are flushed immediately 
        self.terminal.flush()
        self.log.flush()

    def __del__(self):
        self.log.close()  # Ensure the log file is closed when the object is deleted

# Set up device - use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


data pre-processing

In [4]:
# Data augmentation and normalization for training
transform_train = transforms.Compose([
   transforms.RandomCrop(32, padding=4),
   transforms.RandomHorizontalFlip(),
   transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
   transforms.RandomRotation(15),  # Random rotation
   transforms.ToTensor(),
   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Only normalization for testing/validation
transform_test = transforms.Compose([
   transforms.ToTensor(),
   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Set data path variable
data_path = "./data"
full_trainset = datasets.CIFAR10(root=data_path, train=True, download=True, transform=transform_train)

# Dynamically calculate training and validation set sizes
total_size = len(full_trainset)
train_size = int(0.9 * total_size)
val_size = total_size - train_size
trainset, valset = random_split(full_trainset, [train_size, val_size])

# Load test set
testset = datasets.CIFAR10(root=data_path, train=False, download=True, transform=transform_test)

# Create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=100, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# CIFAR10 classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [5]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
   """
   Train and validate the model over specified epochs
   Returns trained model and training history
   """
   since = time.time()
   best_model_wts = copy.deepcopy(model.state_dict())
   best_acc = 0.0
   
   # Initialize history lists for tracking metrics
   train_loss_history = []
   train_acc_history = []
   val_loss_history = []
   val_acc_history = [] 

   for epoch in range(num_epochs):
       print(f'Epoch {epoch}/{num_epochs - 1}')
       print('-' * 10)
       # Each epoch has a training and validation phase
       for phase in ['train', 'val']:
           if phase == 'train':
               model.train() 
               dataloader = trainloader
           else:
               model.eval()  
               dataloader = valloader

           running_loss = 0.0
           running_corrects = 0
           # Iterate over data batches
           for inputs, labels in dataloader:
               inputs = inputs.to(device)
               labels = labels.to(device)

               # Zero the parameter gradients
               optimizer.zero_grad()

               # Forward pass
               # Track history only in train phase
               with torch.set_grad_enabled(phase == 'train'):
                   outputs = model(inputs)
                   _, preds = torch.max(outputs, 1)
                   loss = criterion(outputs, labels)

                   # Backward pass + optimize only in training phase
                   if phase == 'train':
                       loss.backward()
                       optimizer.step()
               # Statistics
               running_loss += loss.item() * inputs.size(0)
               running_corrects += torch.sum(preds == labels.data)

           # Learning rate scheduling
           if phase == 'train':
               scheduler.step()

           # Calculate epoch metrics
           epoch_loss = running_loss / len(dataloader.dataset)
           epoch_acc = running_corrects.double() / len(dataloader.dataset)

           print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

           # Record history
           if phase == 'train':
               train_loss_history.append(epoch_loss)
               train_acc_history.append(epoch_acc.cpu().numpy())
           else:
               val_loss_history.append(epoch_loss)
               val_acc_history.append(epoch_acc.cpu().numpy())

           # Save best model
           if phase == 'val' and epoch_acc > best_acc:
               best_acc = epoch_acc
               best_model_wts = copy.deepcopy(model.state_dict())

   # Print training time and load best model
   time_elapsed = time.time() - since
   print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
   print(f'Best val Acc: {best_acc:4f}')
   model.load_state_dict(best_model_wts)
   
   return model, train_loss_history, train_acc_history, val_loss_history, val_acc_history

def evaluate_model(model, dataloader):
   """
   Evaluate model performance and calculate metrics
   Returns accuracy, precision, F1 score and predictions
   """
   model.eval()
   all_preds = []
   all_labels = []
   
   # Predict without gradient computation
   with torch.no_grad():
       for inputs, labels in dataloader:
           inputs = inputs.to(device)
           labels = labels.to(device)
           outputs = model(inputs)
           _, preds = torch.max(outputs, 1)
           all_preds.extend(preds.cpu().numpy())
           all_labels.extend(labels.cpu().numpy())
   
   # Calculate metrics
   all_preds = np.array(all_preds)
   all_labels = np.array(all_labels)
   accuracy = (all_preds == all_labels).mean()
   precision = precision_score(all_labels, all_preds, average='macro')
   f1 = f1_score(all_labels, all_preds, average='macro')
   
   return accuracy, precision, f1, all_preds, all_labels



In [6]:
def plot_training_history(train_loss, train_acc, val_loss, val_acc, save_path):
   """
   Plot training/validation loss and accuracy curves
   Save the plot to specified path
   """
   plt.figure(figsize=(12, 4))
   
   # Plot loss
   plt.subplot(1, 2, 1)
   plt.plot(train_loss, label='Train Loss')
   plt.plot(val_loss, label='Validation Loss')
   plt.xlabel('Epoch')
   plt.ylabel('Loss')
   plt.legend()
   plt.title('Training and Validation Loss')

   # Plot accuracy
   plt.subplot(1, 2, 2)
   plt.plot(train_acc, label='Train Accuracy') 
   plt.plot(val_acc, label='Validation Accuracy')
   plt.xlabel('Epoch')
   plt.ylabel('Accuracy') 
   plt.legend()
   plt.title('Training and Validation Accuracy')
   
   plt.tight_layout()
   plt.savefig(save_path)
   plt.close()

def plot_confusion_matrix(y_true, y_pred, classes, save_path):
   """
   Create and save confusion matrix heatmap
   """
   cm = confusion_matrix(y_true, y_pred)
   plt.figure(figsize=(10, 8))
   sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
               xticklabels=classes, yticklabels=classes)
   plt.title('Confusion Matrix')
   plt.xlabel('Predicted')
   plt.ylabel('True')
   plt.tight_layout()
   plt.savefig(save_path)
   plt.close()

def print_results(model_name, best_val_acc, test_acc, test_precision, test_f1, train_time):
   """
   Print evaluation metrics in a formatted way
   """
   print(f"Results for {model_name}:")
   print(f"Best Validation Accuracy: {best_val_acc:.4f}")
   print(f"Test Accuracy: {test_acc:.4f}")
   print(f"Test Precision: {test_precision:.4f}")
   print(f"Test F1 Score: {test_f1:.4f}")
   print(f"Training Time: {train_time:.2f} seconds")
   print("-" * 40)

In [7]:
from torchvision.models import resnet18, ResNet18_Weights
def train_resnet18(lr=0.01, momentum=0.9, weight_decay=5e-4, exp_dir=None):
    
   model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
   num_ftrs = model.fc.in_features
   model.fc = nn.Linear(num_ftrs, 10) 
   model = model.to(device)

   # Define loss function and optimizer
   criterion = nn.CrossEntropyLoss()
   optimizer = optim.SGD(model.parameters(), 
                        lr=lr, 
                        momentum=momentum, 
                        weight_decay=weight_decay)
   
   # Learning rate scheduler: reduce the learning rate by a factor of 0.1 every 7 epochs
   scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

   # Train the model and record the time taken
   start_time = time.time()
   model, train_loss, train_acc, val_loss, val_acc = train_model(
       model, criterion, optimizer, scheduler, num_epochs=25)
   end_time = time.time()

   # Plot training curves
   plot_training_history(train_loss, train_acc, val_loss, val_acc, 
                        os.path.join(exp_dir, 'resnet18_training_history.png'))
   
   # Evaluate the model on the test set and print evaluation metrics
   test_acc, test_precision, test_f1, y_pred, y_true = evaluate_model(model, testloader)
   print_results("ResNet18", max(val_acc), test_acc, test_precision, test_f1, 
                end_time - start_time)
   
   # Plot confusion matrix
   plot_confusion_matrix(y_true, y_pred, classes, 
                        os.path.join(exp_dir, 'resnet18_confusion_matrix.png'))


In [8]:
from torchvision.models import vgg16, VGG16_Weights
def train_vgg16(lr=0.01, momentum=0.9, weight_decay=5e-4, exp_dir=None):

   model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
   num_ftrs = model.classifier[6].in_features 
   model.classifier[6] = nn.Linear(num_ftrs, 10) 
   model = model.to(device)

   # Define loss function and optimizer, using the same hyperparameters as ResNet18
   criterion = nn.CrossEntropyLoss()
   optimizer = optim.SGD(model.parameters(), 
                        lr=lr,
                        momentum=momentum, 
                        weight_decay=weight_decay)
   
   # Learning rate scheduler: reduces the learning rate by a factor of 0.1 every 7 epochs
   scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

   # Train the model and record the time taken
   start_time = time.time()
   model, train_loss, train_acc, val_loss, val_acc = train_model(
       model, criterion, optimizer, scheduler, num_epochs=25)
   end_time = time.time()

   # Plot training curves
   plot_training_history(train_loss, train_acc, val_loss, val_acc,
                        os.path.join(exp_dir, 'vgg16_training_history.png'))
   
   # Evaluate the model on the test set and print evaluation metrics
   test_acc, test_precision, test_f1, y_pred, y_true = evaluate_model(model, testloader)
   print_results("VGG16", max(val_acc), test_acc, test_precision, test_f1,
                end_time - start_time)
   
   # Plot confusion matrix
   plot_confusion_matrix(y_true, y_pred, classes,
                        os.path.join(exp_dir, 'vgg16_confusion_matrix.png'))


In [9]:
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
def train_mobilenetv2(lr=0.01, momentum=0.9, weight_decay=5e-4, exp_dir=None):
   
   # Use 'weights' instead of 'pretrained'
   model = models.mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
   num_ftrs = model.classifier[1].in_features 
   model.classifier[1] = nn.Linear(num_ftrs, 10) 
   model = model.to(device)

   # Define loss function and optimizer
   criterion = nn.CrossEntropyLoss()
   optimizer = optim.SGD(model.parameters(), 
                         lr=lr, 
                         momentum=momentum, 
                         weight_decay=weight_decay)
   
   # Learning rate scheduler: reduce LR by 0.1 every 7 epochs
   scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

   # Train the model and record time
   start_time = time.time()
   model, train_loss, train_acc, val_loss, val_acc = train_model(
       model, criterion, optimizer, scheduler, num_epochs=25)
   end_time = time.time()

   # Plot training history
   plot_training_history(train_loss, train_acc, val_loss, val_acc,
                         os.path.join(exp_dir, 'mobilenetv2_training_history.png'))
   
   # Evaluate on test set and print metrics
   test_acc, test_precision, test_f1, y_pred, y_true = evaluate_model(model, testloader)
   print_results("MobileNetV2", max(val_acc), test_acc, test_precision, test_f1,
                 end_time - start_time)
   
   # Plot confusion matrix
   plot_confusion_matrix(y_true, y_pred, classes,
                         os.path.join(exp_dir, 'mobilenetv2_confusion_matrix.png'))


In [10]:
def run_experiments():

   # Create a new experiment directory with a timestamp
   exp_dir = create_experiment_dir()
   
   # Set up logging, saving console output to a file
   sys.stdout = Logger(os.path.join(exp_dir, 'experiment_log.txt'))

   # MobileNetV2 experiments with two sets of hyperparameters
   print("\nStarting MobileNetV2 experiments...")
   # Default settings
   train_mobilenetv2(lr=0.01, momentum=0.9, weight_decay=5e-4, exp_dir=exp_dir)
   # Modified settings with a lower learning rate and higher momentum
   train_mobilenetv2(lr=0.001, momentum=0.95, weight_decay=1e-4, exp_dir=exp_dir)

   # ResNet18 experiments using the same hyperparameter variations
   print("Starting ResNet18 experiments...")
   train_resnet18(lr=0.01, momentum=0.9, weight_decay=5e-4, exp_dir=exp_dir)
   train_resnet18(lr=0.001, momentum=0.95, weight_decay=1e-4, exp_dir=exp_dir)

   # VGG16 experiments using the same hyperparameter variations
   print("\nStarting VGG16 experiments...")
   train_vgg16(lr=0.01, momentum=0.9, weight_decay=5e-4, exp_dir=exp_dir)
   train_vgg16(lr=0.001, momentum=0.95, weight_decay=1e-4, exp_dir=exp_dir)

   # Restore standard output
   sys.stdout = sys.__stdout__


In [11]:
if __name__ == "__main__":
    run_experiments()


Starting MobileNetV2 experiments...
Epoch 0/24
----------
train Loss: 1.4095 Acc: 0.5074
val Loss: 1.1233 Acc: 0.5976
Epoch 1/24
----------
train Loss: 1.0203 Acc: 0.6436
val Loss: 0.9346 Acc: 0.6714
Epoch 2/24
----------
train Loss: 0.8939 Acc: 0.6913
val Loss: 0.9088 Acc: 0.6820
Epoch 3/24
----------
train Loss: 0.8533 Acc: 0.7030
val Loss: 0.8376 Acc: 0.7076
Epoch 4/24
----------
train Loss: 0.8177 Acc: 0.7148
val Loss: 0.7887 Acc: 0.7262
Epoch 5/24
----------
train Loss: 0.8331 Acc: 0.7106
val Loss: 0.8911 Acc: 0.6916
Epoch 6/24
----------
train Loss: 0.7927 Acc: 0.7246
val Loss: 0.8045 Acc: 0.7152
Epoch 7/24
----------
train Loss: 0.6818 Acc: 0.7604
val Loss: 0.6801 Acc: 0.7658
Epoch 8/24
----------
train Loss: 0.6394 Acc: 0.7741
val Loss: 0.6635 Acc: 0.7740
Epoch 9/24
----------
train Loss: 0.6266 Acc: 0.7792
val Loss: 0.6553 Acc: 0.7742
Epoch 10/24
----------
train Loss: 0.6159 Acc: 0.7824
val Loss: 0.6501 Acc: 0.7744
Epoch 11/24
----------
train Loss: 0.6041 Acc: 0.7865
val Lo